In [2]:
import re
from time import sleep
from bs4 import BeautifulSoup
from urllib.request import urlopen

def scrap(url):
    downloaded = urlopen(url).read().decode("utf-8")
    if downloaded:
        return BeautifulSoup(downloaded, "html.parser")
    else:
        raise ValueError('Nothing to scrap')

def store_page(soup, results):
    for product in soup.find_all("div", class_="item-product"):
        description = product.find("div", class_="item-description").find('a')
        item_name = description.find("div", class_="item-name").string
        item_price = " ".join(description.find("span", class_="item-price").string.split())
        item_stock = product.find("span", attrs={"data-store" : re.compile("^stock-product-(\d)+-0$")})
        item_stock = "Infinito" if item_stock is None else "Sin Stock"
        results.append({"item":item_name, "price":item_price, "stock":item_stock})

def store_all_from(base_url, sleep_time=0):
    results = []
    store_page(scrap(base_url), results)

    try:
        page_number = 2
        while True:
            sleep(sleep_time)
            url = f"{base_url}page/{page_number}/?results_only=true&limit=30"
            store_page(scrap(url), results)
            page_number = page_number+1
    except ValueError:
        pass
    finally:
        return results

In [6]:
import json

urls = {
    "almacen":              "https://dieteticarojas.com/almacen/",
    "productos_a_granel":   "https://dieteticarojas.com/productos-a-granel/",
    "salud_y_belleza":      "https://dieteticarojas.com/salud-y-belleza/",
    "frescos_y_congelados": "https://dieteticarojas.com/frescos-y-congelados/",
    "bebidas_e_infusiones": "https://dieteticarojas.com/bebidas-e-infusiones/",
    "apto_vegano":          "https://dieteticarojas.com/apto-veganos/",
    "apto_diabetico":       "https://dieteticarojas.com/apto-diabetico/",
    "aceites_esenciales":   "https://dieteticarojas.com/aceites-esenciales1/",
    "frutas":               "https://dieteticarojas.com/frutas/",
    "productos_organicos":  "https://dieteticarojas.com/productos-organicos/",
    "snacks":               "https://dieteticarojas.com/snacks/"
}

for file_name, url in urls.items():
    results = store_all_from(url, sleep_time=0)
    open(f"results/dietetica_rojas/{file_name}.json", "w").write(json.dumps(results))